In [1]:
ROOT_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl_rebase'
SAVE_DIR = '/gpfs/commons/groups/gursoy_lab/aelhussein/ot_cost/otcost_fl_rebase/results'
import pandas as pd
import sys
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [29]:
def load_results_grad(DATASET):
    with open(f'{ROOT_DIR}/results/{DATASET}/{DATASET}_gradient_diversities_full.pkl', 'rb') as f :
        results = pickle.load(f)
    return results

def load_cost_grad(DATASET):
    results = load_results_grad(DATASET)
    dataset_costs =  pd.read_csv(f'{ROOT_DIR}/results/wasserstein/Wasserstein_{DATASET}.csv').dropna()
    grad_div = [np.mean(results[k]['metric'][-5:-1]) for k in results] # take mean of the final 5 as theres a lot of variance
    dataset_costs['grad_div'] = grad_div
    dataset_costs['dataset'] = DATASET 
    return dataset_costs
def load_all_costs_grad(DATASETS):
    all_datasets_costs = pd.DataFrame()
    for DATASET in DATASETS:
        dataset_costs =  load_cost_grad(DATASET)
        all_datasets_costs = pd.concat([all_datasets_costs, dataset_costs])
    return all_datasets_costs


In [30]:
DATASETS = ['Synthetic', 'Credit', 'Weather',
            'EMNIST', 'CIFAR', 'ISIC', 'IXITiny']
corr_results = {}
for DATASET in DATASETS:
    data = load_cost_grad(DATASET)
    corr_ours, corr_wass = stats.pearsonr(data['grad_div'], data['Ours']), stats.pearsonr(data['grad_div'], data['Wasserstein'])
    corr_results[DATASET]= [corr_ours[0], corr_ours[1], corr_wass[0], corr_wass[1]]
data = load_all_costs_grad(DATASETS)
corr_ours, corr_wass = stats.pearsonr(data['grad_div'], data['Ours']), stats.pearsonr(data['grad_div'], data['Wasserstein'])
corr_results['all']= [corr_ours[0], corr_ours[1], corr_wass[0], corr_wass[1]]

In [36]:
corr_results_df = pd.DataFrame.from_dict(corr_results, orient = 'index', columns = ['Ours', 'Pvalue', 'Wasserstein', 'Pvalue'])
corr_results_df.to_csv(f'{ROOT_DIR}/results/grad_div_corr/grad_div_corr.csv')

In [37]:
corr_results_df

,Ours,Pvalue,Wasserstein,Pvalue
Synthetic,0.830050,0.040870,0.801384,0.055255
Credit,0.924256,0.075744,0.837803,0.162197
Weather,0.761458,0.134740,0.105610,0.865784
EMNIST,0.780360,0.119412,0.303279,0.619857
CIFAR,0.949899,0.050101,0.573981,0.426019
ISIC,0.994456,0.000495,0.974627,0.004833
IXITiny,0.991235,0.084353,0.771917,0.438596
all,0.422412,0.016023,-0.215721,0.235711


In [38]:
#remove the all to just compare datasets
stats.ttest_rel(corr_results_df['Ours'].iloc[:-1], corr_results_df['Wasserstein'].iloc[:-1])

Ttest_relResult(statistic=2.874995457427491, pvalue=0.028241930300463812)